# Hyperparameters Search

One can tune the hyperparameters of a machine learning model using optuna. All that is needed is to define a function that is going to be minimized (or maximized). And the space where the parameters lies. Note: Plots are not showing.

## Introduction

Import the libraries.

In [19]:
import numpy as np
np.random.seed(12345)
import pandas as pd
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
import re

Numpy version: 1.19.5
Pandas version: 1.1.5


In [20]:
import sklearn
print(f"Scikit-Learn version: {sklearn.__version__}")
from sklearn.base import clone
from sklearn.model_selection import KFold

Scikit-Learn version: 0.22.2.post1


In [3]:
def mean_absolute_error(y, y_pred):
    acc = 0
    for idx in range(len(y_pred)):
        error = abs(y[idx]-y_pred[idx])
        acc += error
    acc = acc / len(y_pred)
    return acc
def get_scores(model, X, y, k=10):
    kf = KFold(n_splits=k)
    errors = []
    for train_idx, test_idx in kf.split(X):
        model_ = clone(model)
        model_.fit(X[train_idx], y[train_idx])
        y_pred = model_.predict(X[test_idx])
        err = mean_absolute_error(y[test_idx], y_pred)
        errors.append(err)
    return round(np.mean(errors), 2)

## Prepare dataset

Load the $X$ and $y$ datasets (from drive).

In [4]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
X = pd.read_csv("/content/drive/MyDrive/data-science/solotodo-notebooks-analysis/X.csv")
y = pd.read_csv("/content/drive/MyDrive/data-science/solotodo-notebooks-analysis/y.csv")
X.drop(columns="Unnamed: 0", inplace=True)
y.drop(columns="Unnamed: 0", inplace=True)

Mounted at /content/drive


In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
scaler_pipeline = make_pipeline(StandardScaler(), MinMaxScaler())
X_scaled = scaler_pipeline.fit_transform(X)

## Ridge Tunning

Install optuna to Google Colab

In [6]:
!pip install optuna
import optuna
print(f"Optuna version: {optuna.__version__}")

     |████████████████████████████████| 308 kB 6.3 MB/s 
     |████████████████████████████████| 209 kB 58.2 MB/s 
     |████████████████████████████████| 80 kB 11.1 MB/s 
     |████████████████████████████████| 75 kB 5.1 MB/s 
     |████████████████████████████████| 144 kB 63.9 MB/s 
     |████████████████████████████████| 49 kB 6.6 MB/s 
     |████████████████████████████████| 111 kB 80.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=b85efd4c22c6028d9b93b299fe0d55cc5310607c2d363c568f62b8b370b8b592
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Optuna version: 2.10.0


Define the objective function and optimize it.

Note: both models may have the same random state, to make sure we compare them under the same conditions.

In [7]:
from sklearn.linear_model import Ridge
def ridge_search(trial):
    alpha = trial.suggest_float("alpha", 0, 10)
    fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
    tol = trial.suggest_float("tol", 0, 1)
    random_state = trial.suggest_categorical("random_state", [555])
    model = Ridge(
        alpha=alpha, 
        fit_intercept=fit_intercept, 
        tol=tol, 
        random_state=random_state
    )
    score = get_scores(model, X=X_scaled, y=y.values, k=10)
    return score
ridge_study = optuna.create_study(direction="minimize")

[I 2021-10-14 19:25:37,476] A new study created in memory with name: no-name-40379db8-c104-4d60-94dd-b911f01bb45a


In [8]:
ridge_study.optimize(ridge_search, n_trials=500)

[I 2021-10-14 19:25:37,528] Trial 0 finished with value: 178400.74 and parameters: {'alpha': 9.819608006905868, 'fit_intercept': False, 'tol': 0.367954737447465, 'random_state': 555}. Best is trial 0 with value: 178400.74.
[I 2021-10-14 19:25:37,552] Trial 1 finished with value: 177033.66 and parameters: {'alpha': 4.008888792471018, 'fit_intercept': False, 'tol': 0.6621038007557737, 'random_state': 555}. Best is trial 1 with value: 177033.66.
[I 2021-10-14 19:25:37,577] Trial 2 finished with value: 180647.32 and parameters: {'alpha': 9.234653332988824, 'fit_intercept': True, 'tol': 0.929413971219498, 'random_state': 555}. Best is trial 1 with value: 177033.66.
[I 2021-10-14 19:25:37,602] Trial 3 finished with value: 178241.68 and parameters: {'alpha': 6.229424265577653, 'fit_intercept': True, 'tol': 0.3468997650261939, 'random_state': 555}. Best is trial 1 with value: 177033.66.
[I 2021-10-14 19:25:37,628] Trial 4 finished with value: 178668.94 and parameters: {'alpha': 6.8198707945534

In [21]:
print(ridge_study.best_params)

{'alpha': 3.357007933856045, 'fit_intercept': False, 'tol': 0.411850258814384, 'random_state': 555}


In [22]:
from optuna import visualization
fig = visualization.plot_optimization_history(ridge_study)
fig.show()

There seems to be no more room to a substantive improvement.

## Random Forest Tunning

In [15]:
from sklearn.ensemble import RandomForestRegressor
def rfreg_search(trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)
    criterion = trial.suggest_categorical("criterion", ["mae"])
    max_depth = trial.suggest_int("max_depth", 1, 100)
    min_samples_split = trial.suggest_float("min_samples_split", 0.01, 1)
    min_samples_leaf = trial.suggest_float("min_samples_leaf", 0.01, 0.5)
    min_weight_fraction_leaf = trial.suggest_float("min_weight_fraction_leaf", 0, 0.5)
    max_features = trial.suggest_int("max_features", 1, 13)
    min_impurity_decrease = trial.suggest_float("min_impurity_decrease", 0, 1)
    ccp_alpha = trial.suggest_float("ccp_alpha", 0, 1)
    random_state = trial.suggest_categorical("random_state", [555])
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        min_weight_fraction_leaf=min_weight_fraction_leaf,
        max_features=max_features,
        min_impurity_decrease=min_impurity_decrease,
        ccp_alpha=ccp_alpha,
        random_state=random_state
    )
    score = get_scores(model, X=X.values, y=y.values.ravel(), k=10)
    return score
rfreg_study = optuna.create_study(direction="minimize")

[I 2021-10-14 19:30:04,264] A new study created in memory with name: no-name-39d6bcf9-4e27-4da4-9961-393ca24374b3


In [16]:
rfreg_study.optimize(rfreg_search, n_trials=500)

[I 2021-10-14 19:30:14,744] Trial 0 finished with value: 330686.22 and parameters: {'n_estimators': 405, 'criterion': 'mae', 'max_depth': 83, 'min_samples_split': 0.40544450093789824, 'min_samples_leaf': 0.2772112642226623, 'min_wieght_fraction_leaf': 0.2992450293530424, 'max_features': 11, 'min_impurity_decrease': 0.7448638003457366, 'ccp_alpha': 0.6169018735955101, 'random_state': 555}. Best is trial 0 with value: 330686.22.
[I 2021-10-14 19:30:23,830] Trial 1 finished with value: 378028.22 and parameters: {'n_estimators': 611, 'criterion': 'mae', 'max_depth': 38, 'min_samples_split': 0.6950614704147353, 'min_samples_leaf': 0.37407730027454994, 'min_wieght_fraction_leaf': 0.4948590423445442, 'max_features': 5, 'min_impurity_decrease': 0.7187704833720792, 'ccp_alpha': 0.747185877907856, 'random_state': 555}. Best is trial 0 with value: 330686.22.
[I 2021-10-14 19:30:24,739] Trial 2 finished with value: 371316.61 and parameters: {'n_estimators': 57, 'criterion': 'mae', 'max_depth': 100

In [23]:
print(rfreg_study.best_params)

{'n_estimators': 508, 'criterion': 'mae', 'max_depth': 65, 'min_samples_split': 0.024371138348970756, 'min_samples_leaf': 0.010076361444197107, 'min_wieght_fraction_leaf': 0.009321734735842187, 'max_features': 8, 'min_impurity_decrease': 0.9324005001492703, 'ccp_alpha': 0.8910237262960391, 'random_state': 555}


In [24]:
fig = visualization.plot_optimization_history(rfreg_study)
fig.show()

It seems to have converged to a value.